
&nbsp;

#  The Time-Reversible &nbsp; A C A &nbsp; Lab

(ACA_lab v. 4.0  2025-11-08)

&nbsp;

### A Jupyter Notebook laboratory for "time-reversible Cellular Automata"
### CSB195, Fall 2025
boris.steipe@utoronto.ca


&nbsp;

---

## What is this?
**ACA Lab** is a hands-on notebook for exploring **elementary cellular automata (ECA)** and for *augmenting* them with your own interventions. You can run classic Wolfram rules for ECAs(Elementary Cellular Automata, Rules 0–255), and add small pieces of code that modify how the system **looks** (appearance) or, if you choose, how it **evolves** (state). But you can also impliment automata that probe larger neighborhoods, emit more than one state, change their state when conditions are right, or consider the past along with the present.

This last feature is what we can use to make time reversibel CAs and this notebook explores the possibilites.

Stephen Wolfram has written about such automatas and their relationship to the Second Law of Tthermodynamics in _A New Kind of Science_ ([in the first part of Chapter 9, p. 433 ff.](https://www.wolframscience.com/nks/p433--the-problems-of-physics/)). 

## What does it do?
- **Core engine (Python/NumPy):** evolves a 1-D ECA (Elementary Cellular Automaton) on a ring or an interval (periodic or fixed boundaries).
- **One World ...:**  A "world" is defined by its width in 1-D, and the number of steps in time that it evolves.
- **... Two planes:**  
  - **State** (binary, 0/1) — this is the underlying reality, the states that are read and changed by the Wolfram rules.   
  - **Appearance** (palette indices) — this defines how the states appear. This plane can be recolored/annotated without changing the state.
- **Functions (Cell 2):** This needs to be run once. Then don't touch it again. It's technical. You probably won't need it to experiment with augmenters.
- **Evolutions (Cell 3 ff.):** Each cell runs a self-contained example, defining the world and any required augmenters, and evolving the world step by step . Once the required number of steps have been reached, the final state and history of the world is visualized.
- **Visualization:**  
  - **Trajectories** are written to an HTML page that is sent to the default browser.This makes them nimble, zoomable, and avoids Jupyter limitations on images. The HTML files are standalone - you can rename them and save them for later.

## How to use the notebook
1. Run **Cell 1** and **Cell 2** to initialize, and define helper functions.
2. Read the text, and run **Cell 3 and subsequent cells** one after another, study the results, and experiment what happens when you change parameters.

---

## Notes

...

---


In [ ]:
# =====================================================================
# CELL 1: Import required libraries, install missing, sanity check
# =====================================================================

import subprocess, sys, importlib, time

# External packages used in this notebook
REQUIRED_PACKAGES = ['numpy', 'matplotlib', 'tqdm']  

print(f"Checking for required packages: {', '.join(REQUIRED_PACKAGES)}...")

for package in REQUIRED_PACKAGES:
    try:
        importlib.import_module(package)
        print(f"'{package}' found.")
    except ImportError:
        print(f"'{package}' not found. Attempting installation...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"'{package}' installed successfully.")
        except Exception as e:
            print(f"ERROR: Could not install '{package}'. Please run 'python -m pip install {package}' manually.")
            raise e

# Final imports (after ensuring availability)
import numpy as np
import matplotlib
matplotlib.use("module://matplotlib_inline.backend_inline")  # inline figures in notebook
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
from IPython.display import display

# Sanity info
print("All dependencies are ready.")
print("NumPy:", np.__version__)
print("Matplotlib:", plt.matplotlib.__version__)

# Simple capability check: filesystem writability
out_dir = Path("./ACA_outputs")
out_dir.mkdir(exist_ok=True)
print("Output directory:", out_dir.resolve())
print("Filesystem writable:", out_dir.exists() and out_dir.is_dir())


In [ ]:
# =====================================================================
# CELL 2: Helper functions 
# =====================================================================

from dataclasses import dataclass, asdict
from typing import Literal, Dict, Any
from pathlib import Path
import numpy as np
import json
import base64
import webbrowser, os


NeighborhoodOrderDoc = "Neighborhood bit ordering is 111,110,101,100,011,010,001,000 (Wolfram canonical)."

def rule_to_bitstring(rule: int) -> str:
    """
    Convert Wolfram ECA rule number (0..255) to an 8-char '0'/'1' bitstring.
    Order: 111,110,101,100,011,010,001,000 (MSB for '111').
    """
    if not (0 <= rule <= 255):
        raise ValueError("Rule must be in [0, 255].")
    return format(rule, '08b')


def bitstring_to_rule(bits: str) -> int:
    """
    Convert an 8-char '0'/'1' bitstring (ordered 111..000) to Wolfram rule int.
    """
    if len(bits) != 8 or any(c not in '01' for c in bits):
        raise ValueError("Bitstring must be 8 chars of '0'/'1'.")
    return int(bits, 2)

# Precompute mapping from neighborhood (3-bit index) to output for speed
def _rule_lookup(rule: int) -> np.ndarray:
    bits = rule_to_bitstring(rule)
    # map order 111..000 to integer indices 7..0
    return np.array([int(ch) for ch in bits], dtype=np.uint8)

def apply_rule(neighborhood: tuple[int, int, int], rule: int) -> int:
    """
    Core ECA transition for a single neighborhood.
    Any non-zero inputs are treated as 1.
    """
    l, c, r = (1 if x else 0 for x in neighborhood)  # booleanize
    idx = (l << 2) | (c << 1) | r                    # 0..7 for 111..000 is 7..0
    return int(_rule_lookup(rule)[7 - idx])          # flip to align 111..000

def make_lookup(rule: int) -> np.ndarray:
    """Precompute 8-bit lookup (ordered 111,110,...,000)."""
    return _rule_lookup(rule)

def eca_next(row: np.ndarray, boundary: str, lookup: np.ndarray) -> np.ndarray:
    """
    Vectorized one-step ECA update for an entire row.
    Any non-zero value in `row` counts as 1 (True) for rule evaluation.
    Returns uint8 array with values in {0,1}.
    """
    # --- PATCH: booleanize once, then build neighborhoods on the mask ---
    rb = (row != 0).astype(np.uint8)

    if boundary == 'periodic':
        L = np.roll(rb, +1); C = rb; R = np.roll(rb, -1)
    elif boundary == 'fixed':
        W = rb.shape[0]
        L = np.empty_like(rb); L[0]  = 0; L[1:]  = rb[:-1]
        C = rb
        R = np.empty_like(rb); R[-1] = 0; R[:-1] = rb[1:]
    else:
        raise ValueError("Unknown boundary. Use 'periodic' or 'fixed'.")

    # index 0..7 (00b..111b) computed from booleanized bits
    idx = (L << 2) | (C << 1) | R
    # lookup[0] == output for '111', lookup[7] for '000' → use 7-idx
    return lookup[7 - idx].astype(np.uint8)

# (nice-to-have convenience wrapper)
def eca_step(row: np.ndarray, rule: int, boundary: str = 'periodic') -> np.ndarray:
    """Convenience wrapper (uses booleanized evaluation as above)."""
    return eca_next(row, boundary, make_lookup(rule))


@dataclass               # Using the @dataclass "decorator" to simplify the creation of ...
class World:             # ... the class "World".
    width: int
    steps: int           # a state has steps+1 rows (including t0)
    rule: int            # the Wolfram code that makes this world
    boundary: Literal['periodic','fixed'] = 'periodic'
    seed: int | None = None
    init_mode: Literal['single_on_center','random'] = 'single_on_center'
    init_density: float = 0.5
    palette: Dict[int, str] | None = None
    # the two planes
    state: np.ndarray | None = None
    appearance: np.ndarray | None = None
    t_last: int | None = None    # last completed row index (>=0), None if not evolved yet
    
    def __post_init__(self):
        # --- Require a palette (fail fast) ---
        if self.palette is None:
            raise ValueError(
                "World.palette is required. Define PALETTE in Cell 3 and pass it to World(...). "
                "Example: {0:'#FFFFFF', 1:'#427dae'}"
            )
        _validate_palette(self.palette)  # see helper below
        # Eagerly allocate arrays (the state- and appearance- planes)
        if self.state is None:
            self.state = np.zeros((self.steps + 1, self.width), dtype=np.uint8)
        if self.appearance is None:
            self.appearance = np.zeros((self.steps + 1, self.width), dtype=np.int16)

    # --- JSON (de)serialization helpers ---
def save_world_json(world: World, path: str | Path) -> Path:
    # unchanged usage; __post_init__ guarantees arrays exist
    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    with p.open('w', encoding='utf-8') as f:
        json.dump(world.to_json_dict(), f)
    return p

def load_world_json(path: str | Path) -> World:
    with Path(path).open('r', encoding='utf-8') as f:
        d = json.load(f)
    w = World(
        width=int(d['width']),
        steps=int(d['steps']),
        rule=int(d['rule']),
        boundary=d.get('boundary', 'periodic'),
        seed=d.get('seed', None),
        init_mode=d.get('init_mode', 'single_on_center'),
        init_density=float(d.get('init_density', 0.5)),
        palette=d.get('palette', {0: "#FFFFFF", 1: "#427dae", 2: "#ffc107"}),
        t_last=(None if d.get("t_last") is None else int(d["t_last"])),
    )
    # Overwrite with stored arrays
    w.state = np.array(d['state'], dtype=np.uint8)
    w.appearance = np.array(d['appearance'], dtype=np.int16)
    return w

# --- Validators ---
def _validate_palette(palette: dict[int, str]) -> None:
    if not isinstance(palette, dict):
        raise ValueError("Palette must be a dict mapping int -> hex color string.")
    for k, v in palette.items():
        if not isinstance(k, int):
            raise ValueError(f"Palette key {k!r} is not int.")
        if not (isinstance(v, str) and v.startswith('#') and len(v) in (7, 9)):
            raise ValueError(f"Palette value for key {k} must be '#RRGGBB' or '#RRGGBBAA', got {v!r}.")
        try:
            int(v[1:], 16)
        except Exception:
            raise ValueError(f"Palette color {v!r} is not valid hex.")
    # strictly require keys 0 and 1 
    required = {0, 1}
    missing = required - set(palette.keys())
    if missing:
        raise ValueError(f"Palette is missing required indices: {sorted(missing)}")


# --- HTML rendering: write a self-contained viewer and (optionally) open it ---
def render_html_view(world: World, out_html: str | Path) -> Path:
    import time, os

    t0 = time.perf_counter()
    
    # Choose byte width: 1 byte is enough if your palette indices < 256.
    max_index = int(np.max(world.appearance))
    dtype = np.uint8 if max_index < 256 else np.uint16
    appearance_flat = world.appearance.astype(dtype, copy=False).ravel(order="C")
    b64 = base64.b64encode(appearance_flat.tobytes()).decode("ascii")
    t1 = time.perf_counter()
    
    data_small = {
        "dtype": "u8" if dtype is np.uint8 else "u16",
        "width": world.width,
        "rows": world.steps + 1,
        "palette": world.palette,
        "meta": {
            "rule": world.rule,
            "boundary": world.boundary,
            "seed": world.seed,
            "init_mode": world.init_mode,
            "init_density": world.init_density
        }
    }
    html_template = f"""<!doctype html>
<html>
<head>
<meta charset="utf-8">
<title>ACA Trajectory (Rule {world.rule})</title>
<style>
  body {{ margin: 0; font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif; }}
  #bar {{ padding: 8px 12px; display: flex; gap: 12px; align-items: center; border-bottom: 1px solid #ddd; }}
  #info {{ font-size: 14px; opacity: .8; }}
  #canvas {{ display: block; }}
  button, input {{ font-size: 14px; }}
  .mono {{ font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; }}
</style>
</head>
<body>
  <div id="bar">
    <span id="info"></span>
    <label>Scale: <input id="scale" type="number" step="1" min="1" value="4" style="width:60px;"></label>
    <button id="fit">Fit to window</button>
    <button id="square">Square pixels</button>
    <span class="mono">Palette: {json.dumps(world.palette)}</span>
  </div>
  <canvas id="canvas"></canvas>
  <script id="appearance_b64" type="application/octet-stream">
  {b64}
  </script>
<script>
const data = {json.dumps(data_small)};   
const canvas = document.getElementById('canvas');
const ctx = canvas.getContext('2d', {{alpha:false}});
const info = document.getElementById('info');
const scaleInput = document.getElementById('scale');

const rows = data.rows, cols = data.width, palette = data.palette;

// ---- decode base64 -> typed array (flat A of length rows*cols) ----

const b64 = document.getElementById('appearance_b64').textContent.trim();

function b64ToUint8(b64) {{
  const bin = atob(b64);
  const out = new Uint8Array(bin.length);
  for (let i = 0; i < bin.length; i++) out[i] = bin.charCodeAt(i);
  return out;
}}

console.time("decode");
const buf8 = b64ToUint8(b64);
const A = (data.dtype === "u8") ? buf8 : new Uint16Array(buf8.buffer);
console.timeEnd("decode");
console.log("A length:", (data.dtype==="u8"?buf8.length:new Uint16Array(buf8.buffer).length), "expected:", rows*cols);


// ---- palette packed to Uint32 (AABBGGRR in little-endian) ----
function hexToRgba(hex) {{
  hex = String(hex).trim().replace(/^#/, '');
  if (hex.length === 3)  hex = hex.split('').map(c => c+c).join('') + 'FF';
  else if (hex.length === 4) hex = hex.split('').map(c => c+c).join('');
  else if (hex.length === 6) hex += 'FF';
  if (hex.length !== 8) return [0,0,0,255];
  return [
    parseInt(hex.slice(0,2),16), parseInt(hex.slice(2,4),16),
    parseInt(hex.slice(4,6),16), parseInt(hex.slice(6,8),16)
  ];
}}
const maxKey = Math.max(...Object.keys(palette).map(k => parseInt(k,10)));
const pal32 = new Uint32Array(maxKey + 1);
for (const [k,v] of Object.entries(palette)) {{
  const [r,g,b,a] = hexToRgba(v);
  pal32[parseInt(k,10)] = (a<<24)|(b<<16)|(g<<8)|(r<<0);
}}

// ---- 1× base canvas + fast draw (then scale) ----
const base = document.createElement('canvas');
const bctx = base.getContext('2d', {{alpha:true}});
base.width = cols; base.height = rows;

function draw(scale) {{
  const img = bctx.createImageData(cols, rows);
  const buf32 = new Uint32Array(img.data.buffer);
  let p = 0;
  for (let y = 0; y < rows; y++) {{
    const rowOff = y * cols;
    for (let x = 0; x < cols; x++) {{
      buf32[p++] = pal32[A[rowOff + x]] || 0xFF000000;
    }}
  }}
  bctx.putImageData(img, 0, 0);

  canvas.width = cols * scale;
  canvas.height = rows * scale;
  ctx.imageSmoothingEnabled = false;
  ctx.drawImage(base, 0, 0, canvas.width, canvas.height);

  info.textContent = "Rule " + data.meta.rule +
                     " | boundary=" + data.meta.boundary +
                     " | seed=" + data.meta.seed +
                     " | init=" + data.meta.init_mode;
}}


document.getElementById('fit').addEventListener('click', () => {{
  const sX = Math.max(1, Math.floor(window.innerWidth / cols));
  const sY = Math.max(1, Math.floor((window.innerHeight-48) / rows));
  const s = Math.max(1, Math.min(sX, sY));
  scaleInput.value = s;
  draw(s);
}});

document.getElementById('square').addEventListener('click', () => {{
  const s = parseInt(scaleInput.value || '4', 10);
  draw(Math.max(1, s));
}});

scaleInput.addEventListener('change', () => {{
  const s = parseInt(scaleInput.value || '4', 10);
  draw(Math.max(1, s));
}});

console.time("draw");
draw(parseInt(scaleInput.value, 10));
console.timeEnd("draw");


</script>
</body>
</html>"""

    t2 = time.perf_counter()
    
    out_html = Path(out_html)
    out_html.parent.mkdir(parents=True, exist_ok=True)
    out_html.write_text(html_template, encoding='utf-8')

    size_kb = os.path.getsize(out_html) / 1024
    t3 = time.perf_counter()
    
    print(f"[render_html_view] b64: {(t1-t0)*1000:.1f} ms | template: {(t2-t1)*1000:.1f} ms | write: {(t3-t2)*1000:.1f} ms | size: {size_kb:.1f} KB")
    
    return out_html


def open_html(path: str | Path):
    """Try to open the given HTML file in a new browser tab/window (local setups)."""

    import time, webbrowser
    
    url = Path(path).resolve().as_uri()

    t0 = time.perf_counter()
    ok = webbrowser.open_new_tab(url)
    t1 = time.perf_counter()
    print(f"[open_html] launch: {(t1-t0)*1000:.1f} ms | ok={ok} | URL: {url}")
    print("Opened in browser:" if ok else "Could not auto-open; open manually:", url)

# --- Utility: prettify a rule ---
def describe_rule(rule: int) -> str:
    bits = rule_to_bitstring(rule)
    return f"Rule {rule} (bits {bits}) | {NeighborhoodOrderDoc}"

def parse_grid(g) -> np.ndarray:
    """
    Accepts:
      - "1011"  (1×W string)
      - "10\\n11" (H×W string, bottom line is 'now')
      - np.ndarray[int] (H×W or (W,) -> coerced to (1×W))
    Returns: int ndarray of shape (H, W) with values as given (no coercion to {0,1}).
    """
    if isinstance(g, np.ndarray):
        arr = g.astype(int, copy=False)
        if arr.ndim == 1:
            arr = arr[np.newaxis, :]
        elif arr.ndim != 2:
            raise ValueError("Grid ndarray must be 1D or 2D.")
        return arr
    if not isinstance(g, str):
        raise ValueError("Grid must be a string or numpy array.")
    # Split lines; ignore empty lines; bottom line is 'now'
    rows = [ln.strip() for ln in g.strip().splitlines() if ln.strip() != ""]
    if len(rows) == 1 and all(ch in "0123456789" for ch in rows[0]):
        data = [[int(ch) for ch in rows[0]]]
    else:
        data = [[int(ch) for ch in ln] for ln in rows]
    # Validate rectangular
    Wset = {len(r) for r in data}
    if len(Wset) != 1:
        raise ValueError("All grid rows must have same width.")
    return np.array(data, dtype=int)

def _candidate_anchors(width: int, patch_width: int, periodic: bool) -> np.ndarray:
    if periodic:
        return np.arange(width, dtype=int)
    # non-periodic: only anchors fully inside the segment
    max_start = width - patch_width
    if max_start < 0:
        return np.empty(0, dtype=int)
    return np.arange(max_start + 1, dtype=int)

def _slice_x(arr_row: np.ndarray, x0: int, w: int, periodic: bool) -> np.ndarray:
    """Return arr_row[x0:x0+w] with wrap if periodic; caller guarantees bounds otherwise."""
    W = arr_row.shape[0]
    if periodic:
        idx = (np.arange(x0, x0 + w) % W)
        return arr_row[idx]
    else:
        # out-of-bounds anchors must not be produced upstream
        return arr_row[x0:x0 + w]

def match_at_t(world, plane: str, t: int, pattern: np.ndarray) -> np.ndarray:
    """
    Find anchors x0 where pattern (H×W) matches rows [t-H+1 ... t] on the given plane.
    Inherits wrap from world.boundary. Returns int array of anchors (possibly empty).
    """
    if plane not in ("state", "appearance"):
        raise ValueError("plane must be 'state' or 'appearance'")
    pat = parse_grid(pattern)
    H, Wp = pat.shape
    if t + 1 < H:
        return np.empty(0, dtype=int)

    A = getattr(world, plane)  # shape (steps+1, width)
    width = world.width
    periodic = (world.boundary == 'periodic')

    anchors = _candidate_anchors(width, Wp, periodic)
    if anchors.size == 0:
        return anchors

    # Start with all candidates valid; knock out mismatches row by row
    ok = np.ones(anchors.shape[0], dtype=bool)
    # rows in the world corresponding to pattern rows (top..bottom) = (t-H+1 .. t)
    row_indices = range(t - H + 1, t + 1)

    for h, r in enumerate(row_indices):
        row = A[r]  # (width,)
        # Build window slices for all candidate anchors
        # windows[k] = row slice at anchor k (length Wp)
        if periodic:
            idx = (anchors[:, None] + np.arange(Wp)) % width  # shape (K, Wp)
            windows = row[idx]                                # shape (K, Wp)
            eq = np.all(windows == pat[h], axis=1)            # (K,)
        else:
            # In non-periodic, anchors already in-bounds
            # vectorized gather via broadcasting
            idx = anchors[:, None] + np.arange(Wp)            # (K, Wp)
            windows = row[idx]
            eq = np.all(windows == pat[h], axis=1)
        ok &= eq
        if not ok.any():
            return np.empty(0, dtype=int)
    return anchors[ok]

def match_at_row(world, plane: str, t_ref: int, pattern: np.ndarray) -> np.ndarray:
    """
    Find anchors x0 where pattern (H×W) matches rows [t_ref-H+1 ... t_ref] on plane.
    """
    if plane not in ("state", "appearance"):
        raise ValueError("plane must be 'state' or 'appearance'")
    pat = parse_grid(pattern)
    H, Wp = pat.shape
    if t_ref + 1 < H:
        return np.empty(0, dtype=int)

    A = getattr(world, plane)
    width = world.width
    periodic = (world.boundary == 'periodic')

    anchors = _candidate_anchors(width, Wp, periodic)
    if anchors.size == 0:
        return anchors

    ok = np.ones(anchors.shape[0], dtype=bool)
    for h, r in enumerate(range(t_ref - H + 1, t_ref + 1)):
        row = A[r]
        if periodic:
            idx = (anchors[:, None] + np.arange(Wp)) % width
            windows = row[idx]
        else:
            idx = anchors[:, None] + np.arange(Wp)
            windows = row[idx]
        ok &= np.all(windows == pat[h], axis=1)
        if not ok.any():
            return np.empty(0, dtype=int)
    return anchors[ok]

    
def apply_patch(world, plane: str, t: int, x0: int, patch: np.ndarray) -> None:
    """
    Overwrite rows [t-H+1 ... t], columns [x0 ... x0+W-1] with 'patch' (H×W).
    Wraps horizontally if world.boundary == 'periodic'. Fail-fast on invalid palette indices.
    """
    if plane not in ("state", "appearance"):
        raise ValueError("plane must be 'state' or 'appearance'")
    P = parse_grid(patch)
    H, Wp = P.shape
    if t + 1 < H:
        return  # not enough history yet
    A = getattr(world, plane)
    width = world.width
    periodic = (world.boundary == 'periodic')

    # Appearance palette coverage (fail-fast)
    if plane == "appearance":
        palette_keys = set(world.palette.keys())
        used = set(int(v) for v in np.unique(P))
        if not used.issubset(palette_keys):
            missing = sorted(used - palette_keys)
            raise ValueError(f"Appearance patch uses indices not in palette: {missing}")

    # Write row by row (vectorized along x via index arrays)
    for h, r in enumerate(range(t - H + 1, t + 1)):
        if periodic:
            idx = (np.arange(x0, x0 + Wp) % width)
            A[r, idx] = P[h]
        else:
            if x0 < 0 or x0 + Wp > width:
                # out-of-bounds anchors should not have been produced
                continue
            A[r, x0:x0 + Wp] = P[h]


# ---- World factory (fresh world from config) ----
def init_world() -> World:
    """(Re)initialize the World using globals defined in Cell 3."""
    global WIDTH, STEPS, RULE, BOUNDARY, RANDOM_SEED, INIT_MODE, INIT_DENSITY, PALETTE

    world = World(width=WIDTH, steps=STEPS, rule=RULE, boundary=BOUNDARY,
                  seed=RANDOM_SEED, init_mode=INIT_MODE,
                  init_density=INIT_DENSITY, palette=PALETTE)

    rng = np.random.default_rng(RANDOM_SEED) if RANDOM_SEED is not None else np.random.default_rng()

    if INIT_MODE == 'single_on_center':
        t0 = np.zeros(WIDTH, dtype=np.uint8)
        t0[WIDTH // 2] = 1
    elif INIT_MODE == 'random':
        if not (0.0 <= INIT_DENSITY <= 1.0):
            raise ValueError("INIT_DENSITY must be in [0,1].")
        t0 = (rng.random(WIDTH) < INIT_DENSITY).astype(np.uint8)
    else:
        raise ValueError(f"Unknown INIT_MODE: {INIT_MODE}")

    world.state[0] = t0
    world.appearance[0] = t0.astype(np.int16)
    world.t_last = 0

    print(describe_rule(RULE))
    print(f"World initialized: width={WIDTH}, steps={STEPS}, boundary={BOUNDARY}, "
          f"init={INIT_MODE}, seed={RANDOM_SEED}")
    return world


# ---- Helper to manually create inputs ----
def string_2_int(s: str) -> np.ndarray:
    """
    Convert a string like "020122" (ignoring spaces/underscores) into
    a uint8 NumPy vector: array([0,2,0,1,2,2], dtype=uint8).
    Raises on any non-digit character.
    """
    s_clean = s.replace(" ", "").replace("_", "")
    if not s_clean.isdigit():
        bad = [c for c in s_clean if not c.isdigit()]
        raise ValueError(f"Non-digit characters in input: {bad!r}")
    return np.fromiter((int(ch) for ch in s_clean), dtype=np.uint8)



# ---- Prepare augmenters, merging pal_add with optional recolor ----
def prepare_augmenters(world: World, augs: list[dict]) -> list[dict]:
    prepped, additions = [], {}
    for i, spec in enumerate(augs):
        if not spec.get("enable", True):
            continue

        # palette additions (as before)
        for k, v in (spec.get("pal_add") or {}).items():
            k, v = int(k), str(v)
            if k in additions and additions[k] != v:
                raise ValueError(f"Augmenter {i} conflicts on palette[{k}]")
            additions[k] = v

        # normalize apply to a list
        applies = spec.get("apply")
        if isinstance(applies, dict):
            applies = [applies]
        if not isinstance(applies, list) or not applies:
            raise ValueError(f"Augmenter {i}: 'apply' must be a dict or nonempty list of dicts.")

        # parse grids & offsets
        m = spec["match"]
        pat = parse_grid(m["grid"])
        m_row_off = int(m.get("row_offset", 0))

        parsed_applies = []
        for a in applies:
            trg = a["grid"]
            if isinstance(trg, str) and len(trg) >= 2 and trg[0] in "+-" and trg[1:].isdigit():
                appl = pat + int(trg)
            else:
                appl = parse_grid(trg)
            if appl.shape != pat.shape and appl.shape != (1,1):
                # allow 1×1 “paint here” patches; otherwise shapes must match
                raise ValueError(f"Augmenter {i}: pattern/apply shape mismatch: {pat.shape} vs {appl.shape}")
            parsed_applies.append({
                "plane": a["plane"],
                "grid": appl,
                "row_offset": int(a.get("row_offset", 0)),
                "dx": int(a.get("x_offset", 0)), 
            })

        prepped.append({
            "match_plane": m["plane"],
            "pattern": pat,
            "match_row_offset": m_row_off,
            "applies": parsed_applies,
        })

    if additions:
        merged = {**world.palette, **additions}
        _validate_palette(merged)
        for k, v in additions.items():
            if k in world.palette and world.palette[k] != v:
                raise ValueError(f"Palette[{k}] already {world.palette[k]!r}, not {v!r}.")
        world.palette.update(additions)

    return prepped

def run_augmenters(world: World, t_now: int, prepared: list[dict]) -> None:
    for spec in prepared:
        t_match = t_now + spec.get("match_row_offset", 0)
        anchors = match_at_row(world, spec["match_plane"], t_match, spec["pattern"])
        if anchors.size == 0:
            continue
        periodic = (world.boundary == 'periodic')
        W = world.width
        for x0 in anchors.tolist():
            for ap in spec["applies"]:
                t_apply = t_now + ap.get("row_offset", 0)
                dx = ap.get("dx", 0)
                x_apply = (x0 + dx) % W if periodic else (x0 + dx)
                # (Optional) guard fixed boundary writes that fall off the edge
                if not periodic and (x_apply < 0 or x_apply >= W):
                    continue
                apply_patch(world, ap["plane"], t_apply, x_apply, ap["grid"])  


# --- The main method that evolves the world ---
def evolve(
    world: World,
    show_progress: bool = False,
    t_start: int = 0,
    t_stop: int | None = None,  # if None: use world.steps
) -> World:
    """
    Evolve from row t_start to t_stop (INCLUSIVE on the written row).
    Concretely, the loop runs t = t_start..(t_stop-1) and writes row t+1 each time.
    Requirements:
      - world.state[t_start] must already be initialized.
      - If you rely on "appearance" as read-only snapshot, ensure it's mirrored at t_start.
    """
    if t_stop is None:
        t_stop = world.steps

    # --- sanity checks ---
    if not (0 <= t_start < t_stop <= world.steps):
        raise ValueError(f"Require 0 <= t_start < t_stop <= {world.steps}")
    # ensure base row exists
    if world.state is None or world.state.shape[0] < t_start + 1:
        raise ValueError("world.state[t_start] is not initialized.")
    # keep snapshot discipline if you use appearance as read-plane
    if world.appearance is not None:
        world.appearance[t_start] = world.state[t_start].astype(np.int16)

    lookup = make_lookup(world.rule)
    prepared_augs = prepare_augmenters(world, AUGMENTERS)

    it = range(t_start, t_stop)  # writes rows t_start+1 .. t_stop
    if show_progress:
        from tqdm.auto import tqdm
        it = tqdm(it, desc=f"Evolving {t_start}->{t_stop}", leave=False)

    last_written = world.t_last if world.t_last is not None else 0
    for t in it:
        prev = world.state[t]
        nxt  = eca_next(prev, world.boundary, lookup)

        world.state[t+1]      = nxt
        world.appearance[t+1] = nxt.astype(np.int16)  # frozen snapshot

        run_augmenters(world, t_now=t+1, prepared=prepared_augs)

        last_written = t+1

    world.t_last = last_written

    print(f"World evolved: t_start={t_start}, t_stop={t_stop}, t_last={world.t_last}")

    return world
    


---

# Time reversible CAs: 1: 2-state ECAs

A time reversible CA is one in which the past can be deduced from the present. Class 1
automata are not time reversible: they go to uniform states (e.g. all white, or all black)
and the past gets lost. But some ECAs are inherently reversible: Rules 15, 51, 85, 170, 204, and 240.

```
Rule 85:   111 110 101 100 011 010 001 000
            0   1   0   1   0   1   0   1
```

Here is an example of Rule 85 evolution

---


In [ ]:
# ================================================================
# CELL 3: Time reversible ECAs: Rule 85
# ================================================================


# Set World Parameters
# ====================
RULE   = 85            # Wolfram code

WIDTH  = 200                     # spatial width (x)
STEPS  = round(WIDTH * 1.4142)   # number of time steps (y)

BOUNDARY = 'periodic'  # 'periodic' or 'fixed'
RANDOM_SEED = 112358   # for reproducible randomness
INIT_MODE = 'random'   # 'single_on_center' | 'random'
INIT_DENSITY = 0.25    # fraction of cells in state 1 after random initialization

PALETTE = {0: "#FFFFFF",   # state 0 -> OFF (white)
           1: "#427dae",   # state 1 -> ON  (steelblue)
          }

# Initialize
# ==========
AUGMENTERS = [ ]
world = init_world()

# Evolve
# ======
t0 = time.perf_counter()
evolve(world, show_progress=True)
print(f"evolve took {(time.perf_counter()-t0):.3f}s")


# Display
# =======
html_path = render_html_view(world, "./ACA_outputs/trajectory.html")
print("HTML written to:", Path(html_path).resolve())

try:
    open_html(html_path)
except Exception as e:
    print("Could not auto-open HTML in browser:", e)


---

# How do we know this is time reversible?

A time-reversible CA does not lose information during its evolution. Therefore its starting state can be perfectly recovered from any current state by running it backwards in time. Some CAs can do this trivially, for example Rule 204:

```
Rule 204:  111 110 101 100 011 010 001 000
            1   1   0   0   1   1   0   0
```
... here, any cell that is `1` remains in the `1` state, regardless of its neighbourhood, and every `0` remains `0`. The evolution is perfectly symmetric in time, and perfectly bland as well. Nothing ever happens.

In general however, **IF** a Rule is reversible - i.e. it is information-preserving -  it is reversed by a complementary rule. For Rule 85, the complementary rule is Rule 15:

```
Rule 85:   111 110 101 100 011 010 001 000
            0   1   0   1   0   1   0   1

Rule 15:   111 110 101 100 011 010 001 000
            0   0   0   0   1   1   1   1
```

We can visually confirm, by looking at the image of the Rule 85 evolution: for any given row, the application of Rule 15 produces the row _above it_, i.e. going backwards in time. 

For demonstrating this in practice, we:

1. setup a world that is underlyingly Rule 0;
2. evolve it with a Rule 85 augmenter for a number of steps;
3. switch to a Rule 15 augmenter and run for the same number of steps.

If this produces the starting state again, we have just traveled backward in time. 

---


In [ ]:
# ================================================================
# CELL 4: Time reversing Rule 85 with Rule 15
# ================================================================

# For this demonstration, we run Rule 15 independently from Rule 85 on state "2" and "3",
# and we  color state "3" aquamarine to show the difference.


# Set World Parameters
# ====================
RULE   = 85             # Turn the underlying rule off

WIDTH  = 200           # spatial width (x)
STEPS  = 304           # number of time steps (y)

BOUNDARY = 'periodic'  # 'periodic' or 'fixed'
RANDOM_SEED = 112358   # for reproducible randomness
INIT_MODE = 'random'   # 'single_on_center' | 'random'
INIT_DENSITY = 0.67    # fraction of cells in state 1 after random initialization

PALETTE = {0: "#FFFFFF",   # state 0 -> OFF for Rule 85 (white)
           1: "#427dae",   # state 1 -> ON  for Rule 85 (steelblue)
           2: "#FFFFFF",   # state 2 -> OFF for Rule 15 (white)
           3: "#51F1A9",   # state 3 -> ON  for Rule 15 (aquamarine)
          }
AUGMENTERS = [ ]


# Initialize
# ==========
AUGMENTERS = [   
    *[
      { # Rule 85:   111 110 101 100 011 010 001 000
        #             0   1   0   1   0   1   0   1  
        "enable": True,
        "match": {"plane": "state", "grid": pat, "row_offset": -1},               # look at t-1
        "apply": {"plane": "state", "grid": "1", "row_offset": 0, "x_offset": 1}, # paint center at t
      }
      for pat in ["110", "100", "010", "000"]  
    ],
    *[
      { # Rule 15 - OFF:   333 332 323 322
        #                  2   2   2   2 
        "enable": True,
        "match": {"plane": "state", "grid": pat, "row_offset": -1},               # look at t-1
        "apply": {"plane": "state", "grid": "2", "row_offset": 0, "x_offset": 1}, # paint center at t
      }
      for pat in ["333", "332", "323", "322"]  
    ],
    *[
      { # Rule 15 - ON:   233 232 223 222
        #                  3   3   3   3
        "enable": True,
        "match": {"plane": "state", "grid": pat, "row_offset": -1},               # look at t-1
        "apply": {"plane": "state", "grid": "3", "row_offset": 0, "x_offset": 1}, # paint center at t
      }
      for pat in ["233", "232", "223", "222"]  
    ],
    { # Mirror states "1" to appearance
        "enable": True,
        "match": {"plane": "state", "grid": "1", "row_offset": 0},
        "apply": {"plane": "appearance", "grid": "1", "row_offset": 0},
    },
    { # Mirror states "2" to appearance
        "enable": True,
        "match": {"plane": "state", "grid": "2", "row_offset": 0},
        "apply": {"plane": "appearance", "grid": "2", "row_offset": 0},
    },
    { # Mirror states "3" to appearance
        "enable": True,
        "match": {"plane": "state", "grid": "3", "row_offset": 0},
        "apply": {"plane": "appearance", "grid": "3", "row_offset": 0},
    },
]

world = init_world()

# Run
# ===

# First Evolution
evolve(world, t_start = 0, t_stop = (world.steps - 1) // 2)

# Copy last state and add 2. Every 1 is now a 2, Rule 85 no longer matches
# but Rule 15 does.
world.state[world.t_last + 1] = world.state[world.t_last] + 2
world.appearance[world.t_last + 1] = world.state[world.t_last + 1]

# Second evolution
evolve(world, t_start = world.t_last + 1, t_stop = world.steps - 1)

# Confirm: copy t_0 to t_last + 1 ... should give identical rows
world.state[world.t_last + 1] = world.state[0]
world.appearance[world.t_last + 1] = world.appearance[0]


# Display
# =======
html_path = render_html_view(world, "./ACA_outputs/trajectory.html")
print("HTML written to:", Path(html_path).resolve())

try:
    open_html(html_path)
except Exception as e:
    print("Could not auto-open HTML in browser:", e)

# This is remarkable, but also a bit limited since the six time-reversible
# ECAs are quite regular. Let's look at higher-order CAs instead.


---

# Time reversible higher-order CAs

Wolfram mentions that there are 1,800 time-reversible CAs among the 3**(3**3) three state CAs with a next-neighbour neighbourhood ([p. 436](https://www.wolframscience.com/nks/p436--the-notion-of-reversibility--ebookview/)). Some of these – shown in the figure – have quite interesting behaviour. He labels them by their decimal expansion, which we can reverse engineer:

```
  270361043509
  277206003607
  1123289366095
  1123956776897
  3097483878567 <<<
  3681848058291
```

### The "forward" rule 3097483878567

I inferred the forward rule from its decimal expansion in the figure caption:

An online tool will readily convert a decimal that into ternary (e.g. [here](https://www.rapidtables.com/convert/number/base-converter.html?x=3097483878567&base1=10&base2=3)), or you can ask your AI to write you a function. In this case, the rule converts to:

```
decimal 3097483878567  ->  base-3 101222010010222101101222010
```

Using the canonical order of input states for such an automaton (from 222 -> 000 in this case), we obtain
the expansion of the rule:

```
 222 221 220 212 211 210 202 201 200 122 121 120 112 111 110 102 101 100 022 021 020 012 011 010 002 001 000
  1   0   1   2   2   2   0   1   0   0   1   0   2   2   2   1   0   1   1   0   1   2   2   2   0   1   0
```

Sorting the input patterns by their output states helps to construct the augmenter:

```
 222 220 201 121 102 100 022 020 001
  1   1   1   1   1   1   1   1   1 
  
 212 211 210 112 111 110 012 011 010 
  2   2   2   2   2   2   2   2   2  
  
 221 202 200 122 120 101 021 002 000
  0   0   0   0   0   0   0   0   0
```

Remember, time reversible here does NOT mean the rule itself can run in reverse. It means that the evolution can be reverted back to its starting state **IN PRINCIPLE**, though that may require applying a different rule - the two rules together form a pair. While we can easily implement these rules as two separate augmenters, Wolfram only gave us one of the rules.

### It's "reversing" rule 3863160847143

To get to the reversing rule, I magnified the image in the figure and colored the poorly dim, grey tones in the image green. Then the patterns become clearly visible, and you only need to find all of the inpout triplets **and see what the output is in the row ABOVE**, i.e. backwards in time.

The patterns and their outputs are:

```
 222 221 220 212 211 210 202 201 200 122 121 120 112 111 110 102 101 100 022 021 020 012 011 010 002 001 000 
  1   1   1   2   0   0   0   2   2   1   1   1   0   2   2   2   0   0   1   1   1   0   2   2   2   0   0
```

This ternary number can be converted to decimal for reference:

```
 111200022111022200111022200 -> 3863160847143
``` 
... and the rule's components also can be ordered by output state:

```
222 221 220 122 121 120 022 021 020 
 1   1   1   1   1   1   1   1   1  
 
212 201 200 111 110 102 011 010 002 
 2   2   2   2   2   2   2   2   2  
 
211 210 202 112 101 100 012 001 000
 0   0   0   0   0   0   0   0   0
```

The augmenters themselves are pretty straightforward to write: one emits 1s, one emits 2s, we don't need 
any to emit 0s - and we need two more augmenters to mirror the state-plane to the appearance plane - see below.


### What will you see? What does it mean?

- In the top half of the trajectory, the random initial state evolves through complex patterns.
- We place a white line to visually separates the trajectory that was evolved with the forward rule from the trajectory that was evolved with the reversing Rule.
- We copied the last row above the white line as the first (starting) row below the white line, you see the symmetry.
- The reversing augmenters implement the Rule. The trajectory is just as complex as the forward evolution, but mirrored wherever the forward rule zigs, the reversing rule zags.
- Despite the complex patterns, at the end of the reversal, we have arrived back at the starting configuration of t0. We show that we have retraced the trajectory perfectly to the original starting state by appending the state at t0 to the end of the two world planes. Thus, if the reversal was successful, the last two rows are now identical.

That's what you should see. Pretty neat actually.

---

In [ ]:
# ===================================================================
# CELL 4: Time-reversal of rule 3097483878567 with rule 3863160847143
# ===================================================================


# Step one: Set World Parameters
# ==============================
RULE   = 0             # No actual rule operation needed here

WIDTH  = 449           # spatial width (x)
STEPS  = (WIDTH*2) + 3 # number of time steps we'll use twice the width plus a few extra.

BOUNDARY = 'periodic'  # in time-reversible regimes, 'fixed' boundaries would create irreversible edges.
RANDOM_SEED = 112358   # for reproducible randomness

PALETTE = {0: "#FFFFFF",   # state 0 -> OFF (white)
           1: "#25307A",   # state 1 -> ON  (resolution blue)
           2: "#13A7A2",   # state 2 -> ON  (light sea green)
          }


# Step two: Initialize the world
# ==============================

world = init_world()

# for t0, write random vector from [0, 1, 2]
rng = np.random.default_rng(world.seed)
t0 = rng.integers(0, 3, size=world.width, dtype=np.uint8)  # values from [0,1,2] with p=1/3 each
world.state[0] = t0
world.appearance[0] = t0.astype(np.int16)


# Step three: Define forward Augmenters and evolve for the first half of time steps
# =================================================================================

AUGMENTERS = [
    # Implementing the three-state automaton 101222010010222101101222010
    *[
      { # Output "1"
        #  222 220 201 121 102 100 022 020 001
        #   1   1   1   1   1   1   1   1   1 
        "enable": True,
        "match": {"plane": "state", "grid": pat, "row_offset": -1},               # look at t-1
        "apply": {"plane": "state", "grid": "1", "row_offset": 0, "x_offset": 1}, # paint center at t
      }
      for pat in ["222", "220", "201", "121", "102", "100", "022", "020", "001"]  
    ],
    *[
      { # Output "2"
        #  212 211 210 112 111 110 012 011 010 
        #   2   2   2   2   2   2   2   2   2  
        "enable": True,
        "match": {"plane": "state", "grid": pat, "row_offset": -1},                # look at t-1
        "apply": {"plane": "state", "grid": "2", "row_offset": 0, "x_offset": 1},  # paint center at t
      }
      for pat in ["212", "211", "210", "112", "111", "110", "012", "011", "010"]  
    ],
    
    { # Mirror states "1" to appearance
        "enable": True,
        "match": {"plane": "state", "grid": "1", "row_offset": 0},
        "apply": {"plane": "appearance", "grid": "1", "row_offset": 0},
    },
    
    { # Mirror states "2" to appearance
        "enable": True,
        "match": {"plane": "state", "grid": "2", "row_offset": 0},
        "apply": {"plane": "appearance", "grid": "2", "row_offset": 0},
    },
]

# Forward Evolution
START = time.perf_counter()
evolve(world, t_start = 0, t_stop = (world.steps//2)-2)


# Step four: prepare the reversal
# ===============================

# separate the two evolutions by advancing the process by one (empty) step.
world.t_last += 1

# Copy the last state of the forward evolution:
world.state[world.t_last + 1] = world.state[world.t_last-1]
world.appearance[world.t_last + 1] = world.appearance[world.t_last-1]
world.t_last += 1


# Step five: Define reversing Augmenters and evolve for the second half of time steps
# ====================================================================================

AUGMENTERS = [
    # Implementing the three-state automaton 111200022111022200111022200
    *[
      { # Output "1"
        # 222 221 220 122 121 120 022 021 020 
        #  1   1   1   1   1   1   1   1   1  
        "enable": True,
        "match": {"plane": "state", "grid": pat, "row_offset": -1},               # look at t-1
        "apply": {"plane": "state", "grid": "1", "row_offset": 0, "x_offset": 1}, # paint center at t
      }
      for pat in ["222", "221", "220", "122", "121", "120", "022", "021", "020"]  
    ],
    *[
      { # Output "2"
        # 212 201 200 111 110 102 011 010 002 
        #  2   2   2   2   2   2   2   2   2  
        "enable": True,
        "match": {"plane": "state", "grid": pat, "row_offset": -1},                # look at t-1
        "apply": {"plane": "state", "grid": "2", "row_offset": 0, "x_offset": 1},  # paint center at t
      }
      for pat in ["212", "201", "200", "111", "110", "102", "011", "010", "002"]  
    ],
    
    { # Mirror states "1" to appearance
        "enable": True,
        "match": {"plane": "state", "grid": "1", "row_offset": 0},
        "apply": {"plane": "appearance", "grid": "1", "row_offset": 0},
    },
    
    { # Mirror states "2" to appearance
        "enable": True,
        "match": {"plane": "state", "grid": "2", "row_offset": 0},
        "apply": {"plane": "appearance", "grid": "2", "row_offset": 0},
    },
]

# Reverse evolution
evolve(world, t_start = world.t_last, t_stop = world.t_last + (world.steps//2) - 2)


# For confirmation: copy t_0 to t_last + 1 ... the two rows should be identical
world.state[world.t_last + 1] = world.state[0]
world.appearance[world.t_last + 1] = world.appearance[0]

print(f"Evolution took {(time.perf_counter()-START):.3f}s")

# Step six: Visualize
# ===================

html_path = render_html_view(world, "./ACA_outputs/trajectory.html")
open_html(html_path)



---

# Time reversal of ECAs

Let's think again about time-reversal in general. The 2-D evolution image is a map of a 1-D world (each row) over time. Going down in the imgae means time runs forward, because we always place the next computed row below the current one. Going up in the image means going back in time.

What if we could modify our rules such that going up or going down always undoes what the rule did?

That means: if we flipped a state from `ON` to `OFF`, the reversal should flip it from `OFF` to `ON`. And if the state did not change going forward, than it should not change state either when we go back. It turns out, this can be implemented by applying an `XOR` rule to the previous state ($t_{-1}$) and the resulting state ($t_{+1}$) after applying any ECA Rule to $t_{0}$.

### XOR Truth table

  <table style="margin-left: 2%;">
    <thead>
      <tr>
        <th>$t_{-1}$</th>
        <th>$t_{+1}$</th>
        <th>$t_{+1}$ ($t_{-1} \oplus t_{+1}$)</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>0</td>
        <td>0</td>
        <td>0</td>
      </tr>
      <tr>
        <td>0</td>
        <td>1</td>
        <td>1</td>
      </tr>
      <tr>
        <td>1</td>
        <td>0</td>
        <td>1</td>
      </tr>
      <tr>
        <td>1</td>
        <td>1</td>
        <td>0</td>
      </tr>
    </tbody>
  </table>

... what this means: if the application of a rule flipped a state, we write a `1` instead. and if it left the state the same, we write a `0` instead. I.e. we are now not working with raw states, but with **behaviour** - and behaviour inherently reflects **time**. It is also easy to see that the `XOR` rule makes any output reversible. If you switch the first and the second column (i.e. in the time-reversed order) you get the same _output_ states:

  <table style="margin-left: 2%;">
    <thead>
      <tr>
        <th>$t_{+1}$</th>
        <th>$t_{-1}$</th>
        <th>$t_{+1}$ ($t_{-1} \oplus t_{+1}$)</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>0</td>
        <td>0</td>
        <td>0</td>
      </tr>
      <tr>
        <td>1</td>
        <td>0</td>
        <td>1</td>
      </tr>
      <tr>
        <td>0</td>
        <td>1</td>
        <td>1</td>
      </tr>
      <tr>
        <td>1</td>
        <td>1</td>
        <td>0</td>
      </tr>
    </tbody>
  </table>

To implement this, we simply need an augmenter that looks back in time one step, then behaves accordingly. In the next cell, we run Rule 60, a Class 3 ECA with fractal behaviour on a random initialization row, just to remind ourselves what that looks like. No time reversal yet - Rule 60 is not reversible.




In [ ]:
# ================================================================
# CELL 5: Plain vanilla Rule 60
# ================================================================


# Set World Parameters
# ====================
RULE   = 60     # Wolfram code

WIDTH  = 256   # spatial width (x)
STEPS  = 300   # number of time steps (y)

BOUNDARY = 'periodic'  # 'periodic' or 'fixed'
RANDOM_SEED = 112358   # for reproducible randomness
INIT_MODE = 'random'   # 'single_on_center' | 'random'
INIT_DENSITY = 0.25    # fraction of cells in state 1 after random initialization

PALETTE = {0: "#FFFFFF",   # state 0 -> OFF (white)
           1: "#427dae",   # state 1 -> ON  (steelblue)
           2: "#C2285B",   # marker (rose red)
          }

# Initialize
# ==========
AUGMENTERS = [ ]
world = init_world()

# Evolve
# ======
START = time.perf_counter()
evolve(world, show_progress=True)
print(f"Evolution took {(time.perf_counter()-START):.3f}s")

# Overwrite the last appearance row with all 2s as an end-marker
world.appearance[world.t_last] = np.full(world.width, 2)

# Display
# =======
html_path = render_html_view(world, "./ACA_outputs/trajectory.html")
print("HTML written to:", Path(html_path).resolve())

try:
    open_html(html_path)
except Exception as e:
    print("Could not auto-open HTML in browser:", e)



---

Hold on - did you notice the red line at the bottom of the plot? That was a marker I placed into the final row. But the evolution of the pattern stopped before the last step! In fact this always happens with Rule 60, or its cognate Rule 102, or their respective inverses, Rule 193 and Rule 153 **whenever the world has a width of an integer power of 2**. The evolution develops in triangles or chaotically and then briefly becomes regular and extinguishes after as many steps as the pattern was wide - irrespective of the initial state. Obviously, beyond that point all information about the prior states is irrecoverably lost.

You should try it yourself, just use one of the four rules I mentioned, start from a random state, maybe use a width of 512, 520 steps ... it's pretty wild to see this happen.

Now let's see if we can preserve the information with our time-reversible augmentation.

---


# Implementing XOR as augmenters

In the cell below, we run the same evolution, but add the XOR augmenters. We need two augmenters - one that overwrites the current state with `0` if the current and past states are both `1`, and one that overwrites the current state with `1` if the current state is `0` and the past state was `1` (see the bolded raws bewlow).

  <table style="margin-left: 2%;">
    <thead>
      <tr>
        <th>$t_{-1}$</th>
        <th>$t_{+1}$</th>
        <th>$t_{+1}$ ($t_{-1} \oplus t_{+1}$)</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>0</td>
        <td>0</td>
        <td>0</td>
      </tr>
      <tr>
        <td>0</td>
        <td>1</td>
        <td>1</td>
      </tr>
      <tr>
        <td><b>1</b></td>
        <td><b>0</b></td>
        <td><b>1</b></td>
      </tr>
      <tr>
        <td><b>1</b></td>
        <td><b>1</b></td>
        <td><b>0</b></td>
      </tr>
    </tbody>
  </table>

The only thing we need to be careful about is that the augmenters work consecutively – i.e. the second one would undo the work of the first one. Therefore we do this in the following steps:

1. Apply a standard ECA rule to $t_0$ to give $t_1$.
1. Apply the first augmenter to read from `world.state[now]` and `world.state[before]`, apply the first part of the `XOR` rule wherever it matches, and write the result to the `world.appearance[]` plane.
1. Apply the second augmenter. It also reads from `world.state[]` but that's oK, because we haven't changed it. It too writes its results to `world.appearance[]`.
2. Copy the current `world.appearance[]` back into `world.state[]`.

Let's see how this changes the appearance of Rule 60. (By convention, we call this Rule 60R). 




In [ ]:
# ================================================================
# CELL 6: Rule 60R
# ================================================================


# Set World Parameters
# ====================
RULE   = 60            # Wolfram code

WIDTH  = 256           # spatial width (x)
STEPS  = 300           # number of time steps (y)

BOUNDARY = 'periodic'  # 'periodic' or 'fixed'
RANDOM_SEED = 112358   # for reproducible randomness
INIT_MODE = 'random'   # 'single_on_center' | 'random'
INIT_DENSITY = 0.25    # fraction of cells in state 1 after random initialization

PALETTE = {0: "#FFFFFF",   # state 0 -> OFF (white)
           1: "#427dae",   # state 1 -> ON  (steelblue)
           2: "#C2285B",   # marker (rose red)
          }


# Initialize
# ==========
AUGMENTERS = [ # Implementing XOR across the row to which the rule was applied:
    *[
      { # before=1, now=0  -> now=1: XOR: make now 1 instead
        "enable": True,
        "match": {"plane": "state",      "grid": pat, "row_offset": 0}, # compare at t_now - 2 and now
        "apply": {"plane": "appearance", "grid": "1", "row_offset": 0},  # store result at t_now
      }
      for pat in ["1\n1\n0", "1\n0\n0"]  
    ],
    *[
      { # before=1, now=1  -> now=0: XOR: make now 0 instead
        "enable": True,
        "match": {"plane": "state",      "grid": pat, "row_offset": 0},  # compare at t_now - 2 and now
        "apply": {"plane": "appearance", "grid": "0", "row_offset": 0},  # store result at t_now
      }
      for pat in ["1\n0\n1", "1\n1\n1"]  
    ],

    # Mirror appearance back into state
    { "enable": True,
      "match": {"plane": "appearance", "grid": "0", "row_offset": 0},
      "apply": {"plane": "state",      "grid": "0", "row_offset": 0} },
    { "enable": True,
      "match": {"plane": "appearance", "grid": "1", "row_offset": 0},
      "apply": {"plane": "state",      "grid": "1", "row_offset": 0} },
]

world = init_world()

# Evolve
# ======
START = time.perf_counter()
evolve(world, show_progress=True)
print(f"Evolution took {(time.perf_counter()-START):.3f}s")

# Overwrite the last appearance row with all 2s as an end-marker
world.appearance[world.t_last] = np.full(world.width, 2)


# Display
# =======
html_path = render_html_view(world, "./ACA_outputs/trajectory.html")
print("HTML written to:", Path(html_path).resolve())

try:
    open_html(html_path)
except Exception as e:
    print("Could not auto-open HTML in browser:", e)



---

What happened here is pretty remarkable: the XOR rule preserves all information. In fact there is one row (at where things previously came to an end) that is all `1s`. Such a uniform row would immediately turn a normal ECA fully on or off. Yet, our XOR update in Rule 60R appears to conserve the information: the evolution continues just as complex as before. But is it really reversible?

---

# Reversing an "R" rule

Behaviour in an XORed time-reversible rule is not undone by running a different rule, but by **inverting the arrow of time** itself.

We need to copy a previous state, and then place it after the current state, Then, going forward becomes going backward in time. Not clear? See what we do below.

In the next cell I leave the augmenters intact and change only whatever we might want to update. Then I:

1. run the evolution a few steps past the singularity row;
2. copy the row at `t_last - 1` to `t_last + 1`;
3. and continue the evolution from there.

---



In [ ]:
# ================================================================
# CELL 7: Rule 60R reversed by row swapping
# ================================================================

# Set World Parameters
# ====================
RULE   = 60            # Wolfram code

WIDTH  = 256           # spatial width (x)
STEPS  = 604           # number of time steps (y)

BOUNDARY = 'periodic'  # 'periodic' or 'fixed'
RANDOM_SEED = 112358   # for reproducible randomness
INIT_MODE = 'random'   # 'single_on_center' | 'random'
INIT_DENSITY = 0.25    # fraction of cells in state 1 after random initialization


# Initialize
# ==========
# augmenters were already defined previously

world = init_world()

# Evolve
# ======
START = time.perf_counter()

# Going forward in time
evolve(world, t_start = world.t_last, t_stop = (world.steps//2) - 2)

# Copying the previous row after the last row, also color the inflection row red.
world.state[world.t_last + 1]      = world.state[world.t_last - 1] 
world.appearance[world.t_last + 1] = world.appearance[world.t_last - 1]
world.appearance[world.t_last] *= 2
world.t_last += 1

# Going backward in time
evolve(world, t_start = world.t_last, t_stop = world.t_last + (world.steps//2) - 2)

# Copy the first row into the last row +1, to confirm it is the same, and color it  red
world.appearance[world.t_last] = world.appearance[0] * 2

print(f"Evolution took {(time.perf_counter()-START):.3f}s")

# Display
# =======
html_path = render_html_view(world, "./ACA_outputs/trajectory.html")
print("HTML written to:", Path(html_path).resolve())

try:
    open_html(html_path)
except Exception as e:
    print("Could not auto-open HTML in browser:", e)

---

# Now experiment

The cell below is an exact copy of thew cell above ... you don't have to change about the evolution, you can just experiment with different rules and world sizes. If anything breaks, you can just copy the code from the cell above.

### Rule 37R
In particular, I would like you to have a look at Rule 37R, especially with long evolution times. Stephen Wolfram has claimed [that Rule 37R does not obey the Second Law of Thermodynamics](https://www.wolframscience.com/nks/p453--irreversibility-and-the-second-law-of-thermodynamics/). Which is a rather remarkable claim. Go, follow the link and have a look, and then you can reproduce some of the figures and experiment some more right here in this notebook.

Enjoy!

---


In [ ]:
# ================================================================
# CELL 8: Rule reversal playground
# ================================================================

# Set World Parameters
# ====================
RULE   = 60            # Wolfram code

WIDTH  = 512           # spatial width (x)
STEPS  = 1200           # number of time steps (y)

BOUNDARY = 'periodic'  # 'periodic' or 'fixed'
RANDOM_SEED = 112358   # for reproducible randomness
INIT_MODE = 'random'   # 'single_on_center' | 'random'
INIT_DENSITY = 0.1    # fraction of cells in state 1 after random initialization


# Initialize
# ==========
# augmenters were already defined previously

world = init_world()

# Evolve
# ======
START = time.perf_counter()

# Going forward in time
evolve(world, t_start = world.t_last, t_stop = (world.steps//2) - 2)

# Copying the previous row after the last row, also color the inflection row red.
world.state[world.t_last + 1]      = world.state[world.t_last - 1] 
world.appearance[world.t_last + 1] = world.appearance[world.t_last - 1]
world.appearance[world.t_last] *= 2
world.t_last += 1

# Going backward in time
evolve(world, t_start = world.t_last, t_stop = world.t_last + (world.steps//2) - 2)

# Copy the first row into the last row +1, to confirm it is the same, and color it  red
world.appearance[world.t_last] = world.appearance[0] * 2

print(f"Evolution took {(time.perf_counter()-START):.3f}s")

# Display
# =======
html_path = render_html_view(world, "./ACA_outputs/trajectory.html")
print("HTML written to:", Path(html_path).resolve())

try:
    open_html(html_path)
except Exception as e:
    print("Could not auto-open HTML in browser:", e)